In [2]:
from backend_algo import init_main_officers_template, generate_main_officers_schedule, officer_to_counter_matrix, find_partial_availability, build_officer_schedules, generate_break_schedules, generate_sos_schedule_matrix, greedy_smooth_schedule_beam, greedy_longest_partition_inclusive, get_intervals_from_schedule, max_coverage_paths_inclusive, fill_sos_counter_manning, prefix_non_zero
from backend_algo import get_officer_last_counter_and_empty_counters, update_main_officers_schedule_last_counter,find_empty_rows, slot_officers_matrix_gap_aware, merge_prefixed_matrices, plot_officer_timetable_with_labels, plot_officer_schedule_with_labels, counter_to_officer_schedule
config = {
    'scrollZoom': False,        # Disable zoom via scroll or pinch
    'displayModeBar': False,   # hide the toolbar
    'staticPlot': False        # keep hover interactions active
}
# === user inputs ===
main_officers_reported = '1-18'
report_gl_counters = "4AC1, 8AC11, 12AC21, 16AC31"
sos_timings = '1000-1300, 2000-2200, 1315-1430;2030-2200,1315-1430;2030-2200, 1000-1130;1315-1430;2030-2200, 1200-2200, 1400-1830, 1400-1830, 1630-1830,1330-2200,1800-2030, 1800-2030, 1730-2200, 1730-1900, 1700-1945'
ro_ra_officers = "3RO2100, 11RO1700,15RO2130"


main_officers_template = init_main_officers_template()
main_officers_schedule, reported_officers, valid_ro_ra = generate_main_officers_schedule(main_officers_template, main_officers_reported, report_gl_counters, ro_ra_officers )
counter_matrix_wo_last, counter_no = officer_to_counter_matrix(main_officers_schedule)
officer_last_counter, empty_counters_2030 = get_officer_last_counter_and_empty_counters(
reported_officers, valid_ro_ra, counter_matrix_wo_last)
updated_main_officers_schedule = update_main_officers_schedule_last_counter(
    main_officers_schedule, officer_last_counter, empty_counters_2030)
counter_matrix, counter_no = officer_to_counter_matrix(updated_main_officers_schedule)
counter_w_partial_availability = find_partial_availability(counter_matrix)
officer_names, base_schedules = build_officer_schedules(sos_timings)
all_break_schedules = generate_break_schedules(base_schedules, officer_names)
chosen_schedule_indices, best_work_count, min_penalty = greedy_smooth_schedule_beam(
    base_schedules,None,all_break_schedules,beam_width=20)
sos_schedule_matrix = generate_sos_schedule_matrix(chosen_schedule_indices, all_break_schedules, officer_names)
schedule_intervals_to_officers, schedule_intervals = get_intervals_from_schedule(sos_schedule_matrix)
chains = greedy_longest_partition_inclusive(schedule_intervals)
paths = max_coverage_paths_inclusive(chains)
print("=== best work count ===")
print(best_work_count)
sos_counter_manning = fill_sos_counter_manning(counter_matrix, paths, schedule_intervals_to_officers)
prefixed_counter_matrix = prefix_non_zero(counter_matrix, "M")
empty_rows, partial_empty_rows, partial_empty_rows_index = find_empty_rows(counter_matrix)
prefixed_sos_counter_manning = slot_officers_matrix_gap_aware(schedule_intervals_to_officers, partial_empty_rows)
merged = merge_prefixed_matrices(prefixed_counter_matrix, prefixed_sos_counter_manning)
officer_schedule = counter_to_officer_schedule(merged)
print(merged)
print(merged.shape)
# Example usage:
fig1 = plot_officer_timetable_with_labels(merged)
fig1.show(config=config)
fig2 = plot_officer_schedule_with_labels(officer_schedule)
fig2.show(config=config)


=== best work count ===
[ 2  2  2  2  2  1  1  1  2  2  2  2  2  4  4  4  7  7  7  4  1  1  4  4
  4  4  4  4  4  4  7  7 10 10  7  4  4  4  6  6  6  6  6  7  7  7  7  7]
[8, 37, 26, 16, 35, 25, 15, 5, 34, 24, 14, 4, 33, 23, 13, 3, 32, 22, 12, 2]

Assignment Statistics:
  Partial counters used: 9/11
  New empty counters used: 3
  Total counters used: 12/41
[['M4' 'M4' 'M4' ... 'S1' 'S1' 'S1']
 ['0' '0' '0' ... '0' '0' '0']
 ['0' '0' '0' ... '0' '0' '0']
 ...
 ['M15' 'M15' 'M15' ... 'M14' 'M14' 'M14']
 ['M5' 'M5' 'M5' ... 'M8' 'M8' 'M8']
 ['M1' 'M1' 'M1' ... 'M4' 'M4' 'M4']]
(41, 48)


In [3]:
import numpy as np

def counter_to_officer_schedule(counter_matrix):
    """
    Convert counter_matrix (counters x time slots) to officer_schedule dict.

    Parameters
    ----------
    counter_matrix : np.ndarray
        Shape (num_counters, num_slots)
        Each element = officer_id (e.g., 'M1', 'S2', or '0')

    Returns
    -------
    officer_schedule : dict
        Keys = officer IDs (e.g., 'M1', 'S2')
        Values = list of length num_slots, 
                  each element = counter number assigned at that slot (int), or 0 if not assigned
    """
    num_counters, num_slots = counter_matrix.shape
    officer_schedule = {}

    for counter_idx in range(num_counters):
        for slot in range(num_slots):
            officer_id = counter_matrix[counter_idx, slot]
            if officer_id == '0':
                continue
            if officer_id not in officer_schedule:
                officer_schedule[officer_id] = [0] * num_slots
            officer_schedule[officer_id][slot] = counter_idx + 1  # counter 1-indexed

    prefix_order = {'M': 0, 'S': 1, 'OT': 2}

    def sort_key(k):
        if k.startswith('OT'):
            prefix = 'OT'
            num_part = k[2:]
        else:
            prefix = k[0]
            num_part = k[1:]
        try:
            num_val = int(num_part)
        except ValueError:
            num_val = float('inf')
        return (prefix_order.get(prefix, 99), num_val)

    # Sort and rebuild the dict
    sorted_keys = sorted(officer_schedule.keys(), key=sort_key)
    sorted_schedule = {k: officer_schedule[k] for k in sorted_keys}

    return sorted_schedule #sorted officer schedule


officer_schedule = counter_to_officer_schedule(merged)


In [4]:
fig = plot_officer_schedule_with_labels(officer_schedule)
fig.show(config={
    'scrollZoom': False,        # Disable zoom via scroll or pinch
    'displayModeBar': False,   # hide the toolbar
    'staticPlot': False        # keep hover interactions active
})